In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, floor
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, IndexToString, StandardScaler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.types import DoubleType
from itertools import product

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder\
    .appName("Classification RandomForest")\
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000")\
    .enableHiveSupport()\
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/21 09:34:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[]

In [3]:
clients_immatriculations = spark.sql("SELECT * FROM clients_immatriculations")

### Supprimer les colones inutiles

In [4]:
clients_immatriculations = clients_immatriculations.drop('immatriculation')
clients_immatriculations = clients_immatriculations.drop('couleur')
clients_immatriculations = clients_immatriculations.drop('marque')

clients_immatriculations = clients_immatriculations.drop('puissance')
clients_immatriculations = clients_immatriculations.drop('nbportes')
clients_immatriculations = clients_immatriculations.drop('occasion')
clients_immatriculations = clients_immatriculations.drop('prix')
clients_immatriculations = clients_immatriculations.drop('nbplaces') 
clients_immatriculations = clients_immatriculations.drop('modele') 
clients_immatriculations = clients_immatriculations.drop('longueur')



### OneHotEncoder

In [5]:
indexer_sexe = StringIndexer(inputCol="sexe", outputCol="sexe_index").fit(clients_immatriculations)
clients_immatriculations = indexer_sexe.transform(clients_immatriculations)

encoder_sexe = OneHotEncoder(inputCol="sexe_index", outputCol="sexe_encoded").fit(clients_immatriculations)
clients_immatriculations = encoder_sexe.transform(clients_immatriculations)

In [6]:
indexer_situationfamiliale = StringIndexer(inputCol="situationfamiliale", outputCol="situationfamiliale_index").fit(clients_immatriculations)
clients_immatriculations = indexer_situationfamiliale.transform(clients_immatriculations)

encoder_situationfamiliale = OneHotEncoder(inputCol="situationfamiliale_index", outputCol="situationfamiliale_encoded").fit(clients_immatriculations)
clients_immatriculations = encoder_situationfamiliale.transform(clients_immatriculations)

## Changer des Boolean en Int

In [7]:
clients_immatriculations = clients_immatriculations.withColumn(
    "deuxiemevoiture",
    when(col("deuxiemevoiture") == False, 0)
    .when(col("deuxiemevoiture") == True, 1)
    .otherwise(col("deuxiemevoiture").cast("int"))
)

clients_immatriculations = clients_immatriculations.withColumn(
    "taux_eligible",
    when(col("taux_eligible") == False, 0)
    .when(col("taux_eligible") == True, 1)
    .otherwise(col("taux_eligible").cast("int"))
)

## Classificateur

In [8]:
# Apprentissage des labels
indexer_model = StringIndexer(inputCol="categorie", outputCol="label").fit(clients_immatriculations)

clients_immatriculations = indexer_model.transform(clients_immatriculations)

In [9]:
feature_cols = [col for col in clients_immatriculations.columns if col not in ["categorie", "label", 'sexe', 'situationfamiliale']]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(clients_immatriculations).select("features", "label")

In [10]:
(trainingData, testData) = data.randomSplit([0.8, 0.2], seed=42)

In [11]:
# Configurer le modèle avec les hyperparamètres
rf = RandomForestClassifier(numTrees=10, maxDepth=15, maxBins=64, labelCol="label", featuresCol="features")

# Entraîner le modèle
model = rf.fit(trainingData)

# Spécifier le chemin HDFS
hdfs_path = "hdfs://namenode:9000/user/model/categorie"

# Sauvegarder le modèle dans HDFS
model.write().overwrite().save(hdfs_path)

indexer_model.write().overwrite().save("hdfs://namenode:9000/user/model/indexer_model")
indexer_sexe.write().overwrite().save("hdfs://namenode:9000/user/model/indexer_sexe")
encoder_sexe.write().overwrite().save("hdfs://namenode:9000/user/model/encoder_sexe")

indexer_situationfamiliale.write().overwrite().save("hdfs://namenode:9000/user/model/indexer_situationfamiliale")
encoder_situationfamiliale.write().overwrite().save("hdfs://namenode:9000/user/model/encoder_situationfamiliale")

